<p>
<font size='5' face='Georgia, Arial'>IIC2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
<br>
<font size='1'> Modificado en 2018-1, 2018-2, 2019-2 y 2020-1 por Equipo Docente IIC2233</font>
</p>

## Funciones *built-in* en Python

Existen muchas funciones que vienen implementadas en Python, principalmente con el propósito de simplificar y 
abstraer cálculos que pueden aplicar a objetos de clases distintas (*duck typing*). Pueden revisar todas ellas en la [documentación de funciones](https://docs.python.org/3.7/library/functions.html) de Python. Veamos algunos ejemplos:

### `len`

Retorna el número de elementos que posee un contenedor, como por ejemplo una lista, un diccionario, un *set*, etc.

In [1]:
print(len([3, 4, 1, 5, 5, 2]))
print(len({"nombre": "Juan", "apellido": "Martínez"}))

6
2


La función `len()` aplicada a un objeto en particular (`objeto`) hace un llamado a `objeto.__len__()`. La función `__len__` viene implementada en varias clases de estructuras de datos _built-in_. 

Podemos ver que llamando a `objeto.__len__()` directamente obtenemos el mismo resultado que a través de `len(objeto)`

In [2]:
print([3, 4, 1, 5, 5, 2].__len__())
print({"nombre": "Juan", "apellido": "Martínez"}.__len__())

6
2


También se puede hacer *overriding* del método `__len__`. Supongamos que queremos implementar un tipo especial de lista cuyo método `__len__` retorna el largo de la lista sin considerar los elementos que se repiten:

In [3]:
class MiLista(list):
    """Tipo especial de lista, donde len(lista)
    retorna el largo sin considerar repetidos"""
    
    def __len__(self):
        # Creamos un set con los datos que tenemos
        datos_sin_repetir = set(self)
        
        # Retornamos el largo de este set aprovechando que elimina los repetidos
        return len(datos_sin_repetir)
    
mi_lista = MiLista([1, 2, 3, 4, 5, 6, 6, 7, 7, 7, 7, 2, 2, 3, 3, 1, 1])
print(len(mi_lista))

7


### `__getitem__`

Al definir esta función dentro de una clase, podemos acceder a los elementos mediante algún tipo de índice o llave usando la notación `objeto[valor]`. Esto da paso a dos comportamientos que podemos emular: una secuencia (donde hay un valor después de otro de manera ordenada, como una lista) o un *mapping* (donde hay llaves que permiten acceder a valores, como un diccionario).

En el primer caso, donde queremos que nuestra clase se comporte como una secuencia, el método `__getitem__` debería recibir enteros (`int`) y lanzar la excepción `IndexError` si es que el índice no es válido.

In [4]:
class EnvoltorioString:
    
    def __init__(self, palabra=None):
        self.palabra = palabra
        
    def __getitem__(self, i):
        print(f"Pidiendo el elemento {i}:")
        return self.palabra[i]

envoltorio = EnvoltorioString("Hola-Mundo")
envoltorio[0]
envoltorio[15]

Pidiendo el elemento 0:
Pidiendo el elemento 15:


IndexError: string index out of range

Definir `__getitem__`, para que nuestra clase se comporte como una secuencia, nos permite iterar sobre la estructura mediante un `for`, es decir, el objeto será **iterable**. En este caso, el `for` irá pidiendo los elementos desde el 0 en adelante hasta que se lance una excepción. En nuestro ejemplo, esto nos permite iterar sobre la palabra completa. Al intentar acceder fuera del largo de la palabra que estamos guardando, se lanza una exepción de tipo `IndexError` que detendrá el `for`.

In [5]:
for caracter in envoltorio:
    print(caracter)
    print()

Pidiendo el elemento 0:
H

Pidiendo el elemento 1:
o

Pidiendo el elemento 2:
l

Pidiendo el elemento 3:
a

Pidiendo el elemento 4:
-

Pidiendo el elemento 5:
M

Pidiendo el elemento 6:
u

Pidiendo el elemento 7:
n

Pidiendo el elemento 8:
d

Pidiendo el elemento 9:
o

Pidiendo el elemento 10:


En el caso de que queramos que nuestra clase se comporte como una estructura de tipo *mapping*, podemos usar cualquier tipo de llave. Si una llave es del tipo equivocado, debería lanzarse la excepción `TypeError`, mientras que si es del tipo correcto pero la llave no existe debería lanzarse la excepción `KeyError`.

El siguiente ejemplo es similar al primero, pero las llaves son *strings* de una palabra y el valor retornado es la cantidad de veces que aparece dicho *string* en la palabra. Como habrás notado, ahora nuestra función `__getitem__` debería recibir un *string*, lanzando la excepción `TypeError` si no es así.

In [6]:
class ContadorLetras:
    
    def __init__(self, palabra=None):
        self.palabra = palabra
        
    def __getitem__(self, key):
        print(f"Pidiendo el elemento {key}:")
        # Notar que puedes decir levantar una excepción si la llave no está
        # if key not in self.palabra:
        #     raise KeyError("La letra no está en la palabra")
        return self.palabra.count(key)

contador_letras = ContadorLetras("Hola-Mundo")
contador_letras["o"]

Pidiendo el elemento o:


2

In [7]:
contador_letras[3]

Pidiendo el elemento 3:


TypeError: must be str, not int

Como supondrás, usar un `for` con nuestra implementación no funcionará, ya que al iterar sobre el objeto se utiliza el índice para acceder a los elementos. 

In [8]:
for i in contador_letras:
    print(i)

Pidiendo el elemento 0:


TypeError: must be str, not int

### `reversed`

La función `reversed()` toma una **secuencia** cualquiera como input y retorna **una copia de la secuencia** en orden inverso. También podemos personalizar la función haciendo *overriding* de `__reversed__` en cada clase. 

Si no personalizamos el método `__reversed__`, se usará el *built-in* que iterará usando `__getitem__` y `__len__`. En ese caso, se itera `__len__` veces sobre el objeto usando `__getitem__` hacia atrás.

Por ejemplo, podemos definir un tipo especial de lista que hace *override* de `__reversed__`. En este caso, intercambia la primera mitad con la segunda, en vez de invertir el orden de los elementos.

In [9]:
lista = [1, 2, 3, 4, 5, 6]


class ListaReversaMitad(list):
    
    def __init__(self, *args):
        super().__init__(args)
        
    def __reversed__(self):
        mitad = len(self) // 2
        return self[mitad:] + self[:mitad]
    

for secuencia in lista, ListaReversaMitad(*lista):
    print(f"Clase {type(secuencia).__name__}: ", end="")
    for elemento in reversed(secuencia):
        print(elemento, end=", ")
    print()

Clase list: 6, 5, 4, 3, 2, 1, 
Clase ListaReversaMitad: 4, 5, 6, 1, 2, 3, 


### `enumerate`

`enumerate()` entrega una especie de generador que retorna tuplas, donde el primer objeto en cada tupla es el indice y el segundo es el ítem original. Por ejemplo, si queremos iterar sobre una lista, y necesitamos obtener tanto el índice como su valor, una forma poco *pythonic* de hacer esto sería la siguiente:

In [10]:
lista = ["a","b","c","d"]

for indice in range(len(lista)):
    elemento = lista[indice]
    print(f"{indice}: {elemento}")

0: a
1: b
2: c
3: d


La función `enumerate` nos permite hacer exactamente mismo, pero de una forma más elegante y *pythonic*:

In [11]:
for indice, elemento in enumerate(lista):
    print(f"{indice}: {elemento}")

0: a
1: b
2: c
3: d


Notar que la función `enumerate` retorna un objeto de tipo `enumerate`, que se comporta de manera similar a un generador, por lo que puedes usar `next` para acceder a sus elementos, y eso es lo que permite utilizarlo en un `for`.

### `zip`

Toma dos o más secuencias o iterables y retorna un iterador que entrega tuplas, donde cada tupla está formada por los elementos i-ésimos de cada una de las secuencias o iterables. La cantidad de elementos que retorna este iterador es igual al menor de los largos de las secuencias o iterables.

De manera similar a `enumerate`, `zip` retorna un objeto de tipo `zip` que se comporta de manera similar a un generador, por lo que también se puede usar `next` para acceder a sus elementos. Además de tener los mismos beneficios que un generador regular (ocupa poco espacio y "genera" los elementos a medida que son requeridos).

A modo de ejemplo, consideremos que tenemos una tupla con los *headers* (o nombres de columnas) de un `CSV` y una tupla con los datos de una persona en particular. Queremos obtener una lista con tuplas, donde en cada una aparezca el *header* con su valor:

In [12]:
columnas = ("nombre", "apellido", "email")
persona = ("Juan", "Perez", "jp1@hotmail.com")

list(zip(columnas, persona))

[('nombre', 'Juan'), ('apellido', 'Perez'), ('email', 'jp1@hotmail.com')]

Si ampliamos nuestro ejemplo con una lista de tuplas de personas:

In [13]:
columnas = ("nombre", "apellido", "email")
personas = [
            ("Juan", "Perez", "jp1@hotmail.com"), 
            ("Gonzalo", "Aldunate", "gan@gmail.com"),
            ("Alberto", "Gomez", "agomez@yahoo.com")
           ]

# El asterico simple es para pasar la lista de personas como argumentos separados:
# Si personas = [p1, p2, p3], entonces lo siguiente es equivalente a zip(columnas, p1, p2, p3)
list(zip(columnas, *personas))

[('nombre', 'Juan', 'Gonzalo', 'Alberto'),
 ('apellido', 'Perez', 'Aldunate', 'Gomez'),
 ('email', 'jp1@hotmail.com', 'gan@gmail.com', 'agomez@yahoo.com')]

También recordemos que `zip` sólo tomará la cantidad de elementos del iterable más corto. Si quieres que tome en cuenta la lista más larga puedes ver la función [`zip_longest`](https://docs.python.org/3.7/library/itertools.html#itertools.zip_longest) del paquete `itertools`.

In [14]:
columnas = ("nombre", "apellido", "email")
persona = ("Juan", "Perez", "jp1@hotmail.com", "+56123123??")

list(zip(columnas, persona))

[('nombre', 'Juan'), ('apellido', 'Perez'), ('email', 'jp1@hotmail.com')]

#### `zip` como inversa de sí misma

`zip` en conjunto con el operador `*` (usado para desempacar listas o tuplas a argumentos de una función) puede ser usado como inversa de la operación `zip`.

In [15]:
a = [1, 2, 3, 4]
b = ["a", "b", "c", "d"]

zipped = zip(a, b)
zipped = list(zipped)
print(zipped)

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


In [16]:
unzipped = zip(*zipped)
unzipped = list(unzipped)
print(unzipped)

[(1, 2, 3, 4), ('a', 'b', 'c', 'd')]


En el ejemplo, al ejecutar `zip(*zipped)` el desempaquetamiento entrega los 4 iterables a `zip` **por separado**. Cada uno de estos iterables tiene largo 2. Luego, `zip` concatena todos los primeros elementos `(1, 2, 3, 4)` y luego todos los segundos elementos `('a', 'b', 'c', 'd')`, volviendo a como estaba en un principio.